In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
training = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/142 Final Project/train.csv')
test = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/142 Final Project/test.csv')

In [8]:
#training = pd.read_csv("C:/Users/Alvis/STA142A/train.csv")
#test = pd.read_csv("C:/Users/Alvis/STA142A/test.csv")
#test_labels = pd.read_csv("C:/Users/Alvis/STA142A/test_labels.csv")

print(f"Train Data dimensions are {training.shape}")
#print(f"Train Data dimensions are {test.shape}")

training.head(3)

Train Data dimensions are (159571, 8)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0


In [9]:
# Visualization - https://www.kaggle.com/sasidharturaga/eda-step-wise-preprocess-and-lstm-classifier
import plotly.express as px

df1 = pd.DataFrame(training[training.columns[2:]].sum(axis=0)).reset_index() 
df1["Type of Comment Violation"] = df1["index"]
df1["Number of Comments"] = df1[0]
df1.head(6)

,index,0,Type of Comment Violation,Number of Comments
0,toxic,15294,toxic,15294
1,severe_toxic,1595,severe_toxic,1595
2,obscene,8449,obscene,8449
3,threat,478,threat,478
4,insult,7877,insult,7877
5,identity_hate,1405,identity_hate,1405


In [10]:
df2 = pd.DataFrame(pd.DataFrame(training[training.columns[2:]].sum(axis=1)).reset_index()[0].value_counts())
df2["Number of Sentences"]=df2[0]
df2["Number of Types of Violations"]=df2.index
df2

,0,Number of Sentences,Number of Types of Violations
0,143346,143346,0
1,6360,6360,1
3,4209,4209,3
2,3480,3480,2
4,1760,1760,4
5,385,385,5
6,31,31,6


In [ ]:
#Display of comments 
df1 = df1.sort_values(0, ascending=False) 
fig1 = px.bar(df1, x="Type of Comment Violation", y="Number of Comments", title="No. of comments per label",text="Number of Comments")
fig1.show()

fig2 = px.bar(df2, x="Number of Types of Violations", y="Number of Sentences", title="Number of Comments by Amount of Labels", text="Number of Sentences")
fig2.show()

In [ ]:
from wordcloud import WordCloud
words = ' '.join([text for text in training['comment_text']])

word_cloud = WordCloud(
                       width=1600,
                       height=800,
                       colormap='PuRd', 
                       margin=0,
                       max_words=500, # Maximum numbers of words we want to see 
                       min_word_length=3, # Minimum numbers of letters of each word to be part of the cloud
                       max_font_size=150, min_font_size=30,  # Font size range
                       background_color="white").generate(words)

plt.figure(figsize=(10, 16))
plt.imshow(word_cloud, interpolation="gaussian")
plt.title("Training Comments Word Cloud")
plt.axis("off")
plt.show()

In [ ]:
words = ' '.join([text for text in test['comment_text']])

word_cloud = WordCloud(
                       width=1600,
                       height=800,
                       colormap='BuGn', 
                       margin=0,
                       max_words=500, # Maximum numbers of words we want to see 
                       min_word_length=3, # Minimum numbers of letters of each word to be part of the cloud
                       max_font_size=150, min_font_size=30,  # Font size range
                       background_color="white").generate(words)

plt.figure(figsize=(10, 16))
plt.imshow(word_cloud, interpolation="gaussian")
plt.title("Test Comments Word Cloud")
plt.axis("off")
plt.show()

In [ ]:
# Preprocessing
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')

def preprocess(training, test):
    training['clean'] = training.apply(lambda row: row['comment_text'].replace("\n"," "), axis=1) #removes new line character
    test['clean'] = test.apply(lambda row: row['comment_text'].replace("\n"," "), axis=1)

    #removes urls
    training['clean'] = training.apply(lambda row: re.sub('http://\S+|https://\S+', '', row['clean']), axis=1)
    test['clean'] = test.apply(lambda row: re.sub('http://\S+|https://\S+', '',row['clean']), axis=1)

    #remove all non-alphanumeric values
    training['clean'] = training.apply(lambda row: re.sub('[^A-Za-z0-9\' ]+', '',row['clean']), axis=1)
    test['clean'] = test.apply(lambda row: re.sub('[^A-Za-z0-9\' ]+', '',row['clean']), axis=1)

    #remove stopwords as they occupy major chunk of the vocabulary
    training['clean'] = training['clean'].apply(lambda x: ' '.join([word for word in x.split() if not word in (stop)]))
    test['clean'] = test['clean'].apply(lambda x: ' '.join([word for word in x.split() if not word in (stop)]))

    #removes all additional spaces
    training['clean'] = training.apply(lambda row: re.sub(' +', ' ', row['clean']).strip(), axis=1)
    test['clean'] = test.apply(lambda row: re.sub(' +', ' ', row['clean']).strip(), axis=1)
    
    #lowercase end result
    training['clean'] = training['clean'].str.lower()
    test = test['clean'].str.lower()

preprocess(training, test)
training.head(10)
test.head(10)

In [ ]:
#Bigram analysis - see https://www.kaggle.com/swathi314/toxic-comments-exploratory-analysis-baseline#Toxic-comment-classification
#need to rewrite two functions, gram_analysis and gram_freq to generate result below
for i in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'clean']:
    gram_freq(training,2, i, 'clean')

In [17]:
#Split training dataset into train_data and test_data. Vectorize both train_data and test_data, then build model around 
# train_data to predict values for test_data. Compare predicted test_data values against real test_data values to get an
# accuracy score.

X_train = training['clean']
X_test = test['clean']
Y_train = training['toxic']

tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
tfidf_vectorizer.fit_transform(X_train.values)
        
train_feature_set=tfidf_vectorizer.transform(X_train.values)
test_feature_set=tfidf_vectorizer.transform(X_test.values)

scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
model=scikit_log_reg.fit(train_feature_set, Y_train) #Model build using training data cleaned comments

[LibLinear]

In [18]:
X_train, X_validation = train_test_split(training, random_state = 999) #default is 25% split for validation
Y_train = X_train['toxic']

tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
tfidf_vectorizer.fit_transform(X_train['clean'].values)

train_feature_set=tfidf_vectorizer.transform(X_train['clean'].values)
test_feature_set=tfidf_vectorizer.transform(X_validation['clean'].values)

scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
model = scikit_log_reg.fit(train_feature_set, Y_train) #Model build using training data cleaned comments

Y_predicted = model.predict(test_feature_set) #Y_predicted are the predicted labels from our model
Y_predicted_prob = model.predict_proba(test_feature_set) #proba model shows two columns: 1st column is prob that it is classified as 0, 2nd column is prob that it is classified as 1
#print(proba)

[LibLinear]

In [15]:
# NEW CODE
# for loop to check the accuracy score for each label

for i in range(2,8):
    X_train, X_validation = train_test_split(training, random_state = 999) #default is 25% split for validation
    Y_train = X_train.iloc[:,i]

    tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
    tfidf_vectorizer.fit_transform(X_train['clean'].values)

    train_feature_set=tfidf_vectorizer.transform(X_train['clean'].values)
    test_feature_set=tfidf_vectorizer.transform(X_validation['clean'].values)

    scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
    model = scikit_log_reg.fit(train_feature_set, Y_train) #Model build using training data cleaned comments

    Y_predicted = model.predict(test_feature_set) #Y_predicted are the predicted labels from our model
    Y_predicted_prob = model.predict_proba(test_feature_set) #proba model shows two columns: 1st column is prob that it is classified as 0, 2nd column is prob that it is classified as 1
    
    #Predicted Accuracy score

    Y_validation = X_validation.iloc[:,i]
    print(f"The classification accuracy of the fitted model for the {training.columns[i]} label is {accuracy_score(Y_validation, Y_predicted)}")

[LibLinear]The classification accuracy of the fitted model for the toxic label is 0.9603940540947034
[LibLinear]The classification accuracy of the fitted model for the severe_toxic label is 0.990449452284862
[LibLinear]The classification accuracy of the fitted model for the obscene label is 0.9790439425463113
[LibLinear]The classification accuracy of the fitted model for the threat label is 0.9970922216930288
[LibLinear]The classification accuracy of the fitted model for the insult label is 0.9720502343769584
[LibLinear]The classification accuracy of the fitted model for the identity_hate label is 0.9922793472539042


In [19]:
#NEW CODE
from sklearn.model_selection import KFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

kf = KFold(n_splits = 5)
kf10 = KFold(n_splits = 10)

#comments -> numbers
tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
tfidf_vectorizer.fit_transform(training['clean'].values)

#cleaned complete training set
training_feature_set = tfidf_vectorizer.transform(training['clean'].values)

#logistic regression obj
scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)


In [20]:
print(training['toxic'][6])

1


In [21]:
#NEW CODE
MSE = []
for train_index, valid_index in kf10.split(training_feature_set):
    X_train, X_valid = training_feature_set[train_index], training_feature_set[valid_index]
    Y_train, Y_valid = training['toxic'][train_index], training['toxic'][valid_index]
    
    model = scikit_log_reg.fit(X_train, Y_train) #Model build using training data cleaned comments
    
    # predicted y labels using X validation set
    Y_predicted = model.predict(X_valid)
    #Y_predicted = model.predict_proba(X_valid)
    score = accuracy_score(Y_valid, Y_predicted)
    
    MSE.append(score)
    
np.mean(MSE)

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

0.9601556533101245

In [23]:
def calcClassMSE(FeatureSet, Class):
    MSE = []
    
    for train_index, valid_index in kf10.split(training_feature_set):
        X_train, X_valid = training_feature_set[train_index], training_feature_set[valid_index]
        Y_train, Y_valid = training[Class][train_index], training[Class][valid_index] #labels

        model = scikit_log_reg.fit(X_train, Y_train) #Model build using training data cleaned comments

        # predicted y labels using X validation set
        Y_predicted = model.predict(X_valid)
        #Y_predicted = model.predict_proba(X_valid)
        score = accuracy_score(Y_valid, Y_predicted)

        MSE.append(score)

    return(np.mean(MSE))
    

In [ ]:
calcClassMSE(training_feature_set, "toxic")

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

In [67]:
MSE

[0.960584033086853,
 0.9562574418750391,
 0.9607068997931942,
 0.9592655261013975,
 0.9587015103089553,
 0.9594535313655449,
 0.9600802155793695,
 0.9606442313718118,
 0.9588268471517203,
 0.95932819452278]

In [ ]:
#Predicted Accuracy score
from sklearn.metrics import accuracy_score
Y_validation = X_validation['toxic']

print(accuracy_score(Y_validation, Y_predicted))


In [6]:
#To Do:
- Run Block 121 for the other 5 labels and test accuracy score
- Test for test data? using the labels 
- Bi-gram post-processing data analysis
- Change cross-validation for subsetting data
- Report

SyntaxError: invalid syntax (<ipython-input-6-ab6a1037256d>, line 2)

In [7]:
#References

#Stop words: https://stackabuse.com/removing-stop-words-from-strings-in-python/
#Remove additional spaces: https://stackoverflow.com/questions/1546226/is-there-a-simple-way-to-remove-multiple-spaces-in-a-string
#Classification (Logistic): https://kavita-ganesan.com/news-classifier-with-logistic-regression-in-python/#.YE01E50zZPY